# Boostrapping Curva SOFR

## Configuración

### Librerías

In [71]:
from scipy.optimize import minimize, newton
from datetime import date
import pandas as pd
import numpy as np

from finrisk import QC_Financial_3 as Qcf
import my_functions as myf

### Fecha de Cálculo

In [2]:
fecha_val = Qcf.QCDate(15, 10, 2021)

## Input para la Curva

In [3]:
archivo = 'data/data_curva_sofr.xlsm'

In [4]:
data = pd.read_excel(archivo, dtypes={'start_date': date, 'end_date': date, 'nominal': float})

In [5]:
data.head(10).style.format({'rate': '{:.5%}'})

,bbg ticker,rate,start_date,end_date,tipo,tenor,periodicity,stub_period,settlement_lag,rolling,nominal,amort_is_cashflow,currency,interest_rate
0,SOFRRATE Index,0.05000%,2021-10-15 00:00:00,2021-10-19 00:00:00,TASA,2D,1Y,SHORT_FRONT,0,MOD_FOLLOW,1000000,True,USD,LINACT360
1,USOSFR1Z BGN Curncy,0.05210%,2021-10-19 00:00:00,2021-10-28 00:00:00,TASA,7D,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
2,USOSFR2Z BGN Curncy,0.05240%,2021-10-19 00:00:00,2021-11-04 00:00:00,TASA,14D,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
3,USOSFR3Z BGN Curncy,0.05220%,2021-10-19 00:00:00,2021-11-11 00:00:00,TASA,21D,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
4,USOSFRA BGN Curncy,0.05200%,2021-10-19 00:00:00,2021-11-19 00:00:00,SWAP,1M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
5,USOSFRB BGN Curncy,0.05260%,2021-10-19 00:00:00,2021-12-19 00:00:00,SWAP,2M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
6,USOSFRC BGN Curncy,0.05400%,2021-10-19 00:00:00,2022-01-19 00:00:00,SWAP,3M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
7,USOSFRD BGN Curncy,0.05610%,2021-10-19 00:00:00,2022-02-19 00:00:00,SWAP,4M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
8,USOSFRE BGN Curncy,0.05570%,2021-10-19 00:00:00,2022-03-19 00:00:00,SWAP,5M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360
9,USOSFRF BGN Curncy,0.06020%,2021-10-19 00:00:00,2022-04-19 00:00:00,SWAP,6M,1Y,SHORT_FRONT,2,MOD_FOLLOW,1000000,True,USD,LINACT360


## Construcción de Operaciones

### Patas Fijas

In [6]:
act360 = Qcf.QCAct360()
lin_wf = Qcf.QCLinearWf()
cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2021), 50)
clp = Qcf.QCCLP()
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodo_irregular = Qcf.StubPeriod.SHORTFRONT
es_bono = False

In [7]:
patas_fijas = []
for row in data.itertuples():
    if row.tipo == 'TASA':
        fecha_inicio = Qcf.build_qcdate_from_string(
            row.start_date.date().isoformat())
        fecha_final = Qcf.build_qcdate_from_string(
            row.end_date.date().isoformat())
        fecha_pago = cal.shift(fecha_final, row.settlement_lag)
        tasa = Qcf.QCInterestRate(row.rate, act360, lin_wf)
        fixed_rate_cashflow = Qcf.FixedRateCashflow(
            fecha_inicio,
            fecha_final,
            fecha_pago,
            row.nominal,
            row.nominal,
            row.amort_is_cashflow,
            tasa,
            clp
        )
        leg = Qcf.Leg()
        leg.append_cashflow(fixed_rate_cashflow)
        patas_fijas.append(leg)
    else:
        fecha_inicio = Qcf.build_qcdate_from_string(row.start_date.date().isoformat())
        fecha_final = Qcf.build_qcdate_from_string(row.end_date.date().isoformat())
        fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(
            Qcf.RecPay.RECEIVE,
            fecha_inicio,
            fecha_final,
            bus_adj_rule,
            Qcf.Tenor(row.periodicity),
            periodo_irregular,
            cal,
            row.settlement_lag,
            row.nominal,
            row.amort_is_cashflow,
            Qcf.QCInterestRate(row.rate, act360, lin_wf),
            clp,
            es_bono
        )
        patas_fijas.append(fixed_rate_leg)

In [8]:
myf.leg_as_dataframe(patas_fijas[1], myf.TipoPata.FIJA)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2021-10-19,2021-10-28,2021-11-01,1000000.0,1000000.0,13.025,True,1000013.025,CLP,0.000521,LinAct360


### Patas OIS

Vamos a usar el tipo de cashflow `IcpClpCashflow`.

In [96]:
patas_ois = []
start_date_index = 1.0
end_date_index = 1.0
for row in data.itertuples():
    if row.tipo == 'TASA':
        fecha_inicio = Qcf.build_qcdate_from_string(
            row.start_date.date().isoformat())
        fecha_final = Qcf.build_qcdate_from_string(
            row.end_date.date().isoformat())
        fecha_pago = cal.shift(fecha_final, row.settlement_lag)
        icp_clp_cashflow = Qcf.IcpClpCashflow2(
            fecha_inicio,
            fecha_final,
            fecha_pago,
            -row.nominal,
            -row.nominal,
            row.amort_is_cashflow,
            0.0,
            1.0,
            start_date_index,
            end_date_index
        )
        leg = Qcf.Leg()
        leg.append_cashflow(icp_clp_cashflow)
        patas_ois.append(leg)
    else:
        fecha_inicio = Qcf.build_qcdate_from_string(
            row.start_date.date().isoformat())
        fecha_final = Qcf.build_qcdate_from_string(
            row.end_date.date().isoformat())
        fecha_pago = cal.shift(fecha_final, row.settlement_lag)
        icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp2_leg(
            Qcf.RecPay.PAY,
            fecha_inicio,
            fecha_final,
            bus_adj_rule,
            Qcf.Tenor(row.periodicity),
            periodo_irregular,
            cal,
            2,
            row.nominal,
            row.amort_is_cashflow,
            0.0,
            1.0,
            True
        )
        patas_ois.append(icp_clp_leg)

In [97]:
myf.leg_as_dataframe(patas_ois[4], myf.TipoPata.ICPCLP)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2021-10-19,2021-11-19,2021-11-23,-1000000.0,-1000000.0,True,-1000000.0,CLP,10000.0,10000.0,0.0,-0.0,0.0,1.0,LinAct360


## Función Objetivo

In [11]:
act365 = Qcf.QCAct365()
exp_wf = Qcf.QCContinousWf()

In [12]:
plazos = Qcf.long_vec()
for p in patas_fijas:
    num = p.size()
    fecha = p.get_cashflow_at(num - 1).get_settlement_date()
    plazos.append(fecha_val.day_diff(fecha))

In [78]:
tasas_ini = [0.0 for i in patas_fijas]

In [79]:
vp = Qcf.PresentValue()
fwd = Qcf.ForwardRates()

In [93]:
def obj(tasa, tasas, cual):
    ctasas = Qcf.double_vec()
    for r in tasas:
        ctasas.append(r)
    zcc = Qcf.QCCurve(plazos, ctasas)
    plazo = zcc.get_values_at(cual).tenor
    zcc.set_pair(plazo, float(tasa))
    lin = Qcf.QCLinearInterpolator(zcc)
    zz = Qcf.ZeroCouponCurve(lin, Qcf.QCInterestRate(0.0, act365, exp_wf))
    vp_fix = vp.pv(fecha_val, patas_fijas[cual], zz)
    # print(vp_fix)
    fwd.set_rates_icp_clp_leg(fecha_val, 1.0, patas_ois[cual], zz)
    vp_flot = vp.pv(fecha_val, patas_ois[cual], zz)
    # print(vp_flot)
    return (vp_fix + vp_flot)

In [98]:
obj(0.0, tasas_ini, 4)

24.99115338420961

## Solución

In [100]:
for cual in range(len(patas_fijas)):
    res = newton(obj, 0.0, args=(tasas_ini, cual), tol=1.e-10, disp=True)
    tasas_ini[cual] = res

In [101]:
tasas_ini

[0.000506943036267446,
 0.0005282326710043044,
 0.0005239701596137861,
 0.0005285611968506994,
 0.000521227095912109,
 0.0005324598063808503,
 0.0005467249349781459,
 0.0005681166756035356,
 0.0005628202281282045,
 0.000611396274502991,
 0.0006397352352494379,
 0.0007078347740354871,
 0.000797693869280062,
 0.0009276420440067696,
 0.0010284042889832272,
 0.001169570991439523,
 0.0023674850826336425,
 0.0038658469786089973,
 0.0064293939672413316,
 0.008287229107828531,
 0.009651714623104636,
 0.010781472424751679,
 0.01170148512784094,
 0.012426235529668294,
 0.013010095920516463,
 0.013526257432555755,
 0.014325024121027846,
 0.015115711785147415,
 0.01580216084764192,
 0.015891959664107405,
 0.015769474331163037,
 0.014861488889803064,
 0.013651991276543188]